In [1]:
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from src.consts import *
from src.validation import validate_jumpstart_cube, display_validate_results
from src.coherence import analyze_deck_theme_coherence_enhanced
from src.improve import apply_swap

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

# Load the data files
oracle_df = pd.read_csv('ThePauperCube_oracle_with_pt.csv')
cube_df = pd.read_csv('JumpstartCube_ThePauperCube_ULTIMATE_Final_v3.csv')

# Generated

In [2]:
# # Import the new swap recommendation function

# # Get all deck coherence results first
# from src.coherence import display_coherence_analysis_enhanced
# from src.improve import display_swap_recommendations, find_best_card_swaps_for_deck


# coherence_results = analyze_deck_theme_coherence_enhanced(cube_df, oracle_df)
# print(coherence_results['Green Big Creatures']['overall_coherence'])
# display_coherence_analysis_enhanced(coherence_results)

# # Find the deck with the lowest coherence score for improvement
# worst_deck = min(coherence_results.items(), key=lambda x: x[1]['overall_coherence'])
# deck_to_improve = worst_deck[0]

# print(f"Deck with lowest coherence: {deck_to_improve} (Score: {worst_deck[1]['overall_coherence']:.1f})")

# # Find the best card swaps for this deck
# swap_recommendations = find_best_card_swaps_for_deck(
#     deck_to_improve, 
#     cube_df, 
#     oracle_df, 
#     coherence_results, 
#     num_swaps=2
# )

# # Display the recommendations
# display_swap_recommendations(swap_recommendations)

In [8]:
# You can also analyze a specific deck by name
# Example: Analyze the "Green Big Creatures" deck

from src.process import optimize_deck_coherence

cube_df = optimize_deck_coherence(cube_df=cube_df, oracle_df=oracle_df)

Analyzing deck: Green Ramp

Current coherence: 1.1

Expected themes: Ramp

Deck colors: G

Found 119 candidate cards to consider

...

- **Giant Growth** (Theme Score: 0.0, CMC: 1.0)

### Cards to Add:

- **Tranquil Landscape** (Theme Score: 9.0) - from oracle pool

- **Contaminated Landscape** (Theme Score: 9.0) - from oracle pool

1.9553846153846155
{'expected_themes': ['Ramp'], 'theme_score': 2.8461538461538463, 'theme_matches': [{'card': 'Arbor Elf', 'score': 1, 'themes': ['Ramp']}, {'card': 'Bushwhack', 'score': 5, 'themes': ['Ramp']}, {'card': 'Nyxborn Hydra', 'score': 2, 'themes': ['Ramp']}, {'card': 'Ram Through', 'score': 0, 'themes': ['Ramp']}, {'card': 'Sakura-Tribe Elder', 'score': 8, 'themes': ['Ramp']}, {'card': "Tamiyo's Safekeeping", 'score': 0, 'themes': ['Ramp']}, {'card': 'Treetop Snarespinner', 'score': 1, 'themes': ['Ramp']}, {'card': 'Tuskguard Captain', 'score': 1, 'themes': ['Ramp']}, {'card': 'Voracious Varmint', 'score': 1, 'themes': ['Ramp']}, {'card': 'Wild Growth', 'score': 0, 'themes': ['Ramp']}, {'card': 'Horrific Assault', 'score': 0, 'themes': ['Ramp']}, {'card': 'Tranquil Landscape', 'score': 9, 'themes': ['Ramp']}, {'card': 'Contaminated Landscape', 'score': 9, 'themes': ['Ramp']}], 'color_coherence': 1.0, 'color_issues': [], 'mana_curve_score': 0.9846153846153847, 'mana_curve': 

In [4]:
# display_coherence_analysis_enhanced(analyze_deck_theme_coherence_enhanced(cube_df, oracle_df))

from src.export import export_cube_to_csv


export_cube_to_csv(cube_df, oracle_df, 'JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv')

Exporting cube to JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv...
✅ Successfully exported 390 cards to JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv

📊 Export Summary:
Total cards: 390
Number of decks: 30

Deck breakdown:
  Azorius Evasion/Flying: 13 cards
  Black Aggro: 13 cards
  White Equipment: 13 cards
  White Control: 13 cards
  White Aggro: 13 cards
  Simic Control: 13 cards
  Selesnya Control: 13 cards
  Red Artifacts: 13 cards
  Red Burn: 13 cards
  Rakdos Burn/Damage: 13 cards
  ... and 20 more decks


'JumpstartCube_ThePauperCube_ULTIMATE_Final_v4.csv'